# RAGAS Evaluation

## 참고 자료

- RAGAS 공식 문서: https://docs.ragas.io/

In [10]:
import time
from typing import Any

import pandas as pd
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
from openrouter_llm import create_embedding_model, create_openrouter_llm
from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness

load_dotenv()

True

In [12]:
llm = model = create_openrouter_llm("openai/gpt-4.1", temperature=0)
embeddings = create_embedding_model("openai/text-embedding-3-small")

## 2. 평가 데이터 준비

샘플 데이터를 생성하거나 기존 데이터를 로드합니다.

In [4]:
# 샘플 데이터 생성
sample_data = [
    {
        "question": "RAG 시스템에서 Retriever의 역할은 무엇인가요?",
        "ground_truth": "Retriever는 사용자의 질문에 관련된 문서를 벡터 데이터베이스에서 검색하여 가져오는 역할을 합니다.",
        "contexts": [
            "RAG 시스템의 Retriever 컴포넌트는 질문과 관련된 문서를 벡터 DB에서 찾아 반환합니다.",
            "벡터 검색은 임베딩 모델을 사용하여 의미적 유사도를 계산합니다.",
        ],
        "answer": "Retriever는 벡터 데이터베이스에서 질문과 관련된 문서를 검색하여 제공하는 컴포넌트입니다.",
    },
    {
        "question": "LangChain의 주요 장점은 무엇인가요?",
        "ground_truth": "LangChain은 LLM 애플리케이션 개발을 위한 통합 프레임워크로, 다양한 LLM과 툴을 쉽게 연결할 수 있습니다.",
        "contexts": [
            "LangChain은 여러 LLM 프로바이더를 통합하여 사용할 수 있는 프레임워크입니다.",
            "최근 LangChain 은 버전이 1.0 이 되면서 많은 패러다임의 변화를 거치게 되었습니다.",
        ],
        "answer": "LangChain의 주요 장점은 다양한 LLM과 도구들을 손쉽게 연결하고 체인으로 구성할 수 있다는 점입니다.",
    },
    {
        "question": "Embedding 모델의 차원은 무엇을 의미하나요?",
        "ground_truth": "Embedding 차원은 텍스트를 벡터로 변환할 때의 벡터 크기를 의미하며, 일반적으로 높을수록 표현력이 좋지만 계산 비용이 증가합니다.",
        "contexts": [
            "Embedding 차원이 높으면 더 세밀한 의미를 표현할 수 있습니다.",
            "OpenAI의 text-embedding-3-large는 3072 차원을 사용합니다.",
        ],
        "answer": "차원은 벡터의 크기를 나타내며, 높을수록 의미를 더 정밀하게 표현하지만 메모리와 계산 비용이 늘어납니다.",
    },
]

pd.DataFrame(sample_data)[["question", "answer"]].head()

,question,answer
0,RAG 시스템에서 Retriever의 역할은 무엇인가요?,Retriever는 벡터 데이터베이스에서 질문과 관련된 문서를 검색하여 제공하는 컴...
1,LangChain의 주요 장점은 무엇인가요?,LangChain의 주요 장점은 다양한 LLM과 도구들을 손쉽게 연결하고 체인으로 ...
2,Embedding 모델의 차원은 무엇을 의미하나요?,"차원은 벡터의 크기를 나타내며, 높을수록 의미를 더 정밀하게 표현하지만 메모리와 계..."


## 3. 데이터 전처리

RAGAS 평가를 위해 contexts를 정규화합니다.

In [5]:
def normalize_contexts(contexts: Any) -> list[str]:
    """컨텍스트 정규화 (dict list → str list)"""
    if not contexts:
        return []

    normalized = []
    for ctx in contexts:
        if isinstance(ctx, str):
            normalized.append(ctx)
        elif isinstance(ctx, dict):
            text = ctx.get("text") or ctx.get("preview") or ""
            if text:
                normalized.append(text)
    return normalized


# 정규화 테스트
test_contexts = sample_data[0]["contexts"]
normalized = normalize_contexts(test_contexts)
print(f"원본: {len(test_contexts)}개")
print(f"정규화 후: {len(normalized)}개")
print(f"\n첫 번째 컨텍스트: {normalized[0][:50]}...")

원본: 2개
정규화 후: 2개

첫 번째 컨텍스트: RAG 시스템의 Retriever 컴포넌트는 질문과 관련된 문서를 벡터 DB에서 찾아 반환...


## 3.5. WebSearch 통합 (실시간 컨텍스트 수집)

**목표**: Tavily/DuckDuckGo를 사용하여 실시간으로 웹에서 정보를 검색하고 RAG 파이프라인을 구축합니다.

**파이프라인**:
1. 사용자 질문 입력
2. WebSearch로 관련 컨텍스트 수집 (Tavily → DuckDuckGo fallback)
3. LLM으로 답변 생성
4. RAGAS로 평가

**커리큘럼 대응**: #1 목적에 맞는 Evaluation Dataset 구축, #2 RAGAS 평가

In [6]:
tavily_tool = TavilySearch(
    max_results=5,
    search_depth="advanced",
    include_raw_content=True,
)

In [15]:
def search_with_fallback(query: str, max_results: int = 5) -> list[str]:
    """Tavily → DuckDuckGo Fallback 검색

    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수

    Returns:
        검색된 컨텍스트 리스트
    """

    # 1. Tavily 시도
    if tavily_tool:
        try:
            results = tavily_tool.invoke({"query": query})
            contexts = [r for r in results]

            if contexts:
                return contexts[:max_results]

        except Exception as e:
            print(f"Tavily 실패: {e}")

### RAGAS 평가 파이프라인 구성

질문 → WebSearch → LLM 답변 생성 → RAGAS 평가 전체 흐름을 실행합니다.

In [22]:
# 평가 파이프라인
def realtime_rag_evaluation(question: str, reference: str = None) -> dict:
    """실시간 RAG 평가 파이프라인

    Args:
        question: 사용자 질문
        reference: Ground truth (선택 사항)

    Returns:
        평가 결과 dict
    """
    # 1. WebSearch로 컨텍스트 수집
    contexts = search_with_fallback(question, max_results=3)

    if not contexts:
        return {"error": "검색 실패"}

    # 2. LLM으로 답변 생성
    context_text = "\n\n".join([f"[Context {i + 1}]\n{ctx}" for i, ctx in enumerate(contexts)])

    prompt = f"""다음 컨텍스트를 참고하여 질문에 답변하세요.
컨텍스트에 없는 정보는 사용하지 마세요.

컨텍스트:
{context_text}

질문: {question}

답변:"""

    response = llm.invoke(prompt)
    answer = response.content

    # 3. RAGAS 평가
    sample = SingleTurnSample(
        user_input=question,
        response=answer,
        retrieved_contexts=contexts,
        reference=reference,
    )

    dataset = EvaluationDataset(samples=[sample])

    # reference 유무에 따라 메트릭 선택
    # faithfulness, answer_relevancy: reference 불필요
    # context_precision, context_recall: reference 필수
    if reference:
        metrics = [faithfulness, answer_relevancy, context_precision, context_recall]
    else:
        metrics = [faithfulness, answer_relevancy]
        print("reference가 없어 context_precision과 context_recall은 평가하지 않습니다.")

    result = evaluate(
        llm=llm,
        embeddings=embeddings,
        dataset=dataset,
        metrics=metrics,
    )

    scores = result.to_pandas().iloc[0]

    print("RAGAS 점수:")
    print(f"  - Faithfulness: {scores.get('faithfulness', 0):.3f}")
    print(f"  - Answer Relevancy: {scores.get('answer_relevancy', 0):.3f}")
    if reference:
        print(f"  - Context Precision: {scores.get('context_precision', 0):.3f}")
        print(f"  - Context Recall: {scores.get('context_recall', 0):.3f}")

    return {
        "question": question,
        "contexts": contexts,
        "answer": answer,
        "scores": {
            "faithfulness": scores.get("faithfulness", 0),
            "answer_relevancy": scores.get("answer_relevancy", 0),
            "context_precision": scores.get("context_precision", 0) if reference else None,
            "context_recall": scores.get("context_recall", 0) if reference else None,
        },
    }

In [23]:
# 테스트: 실시간 평가 실행
test_question = "RAGAS 평가 프레임워크의 핵심 지표는 무엇인가요?"

result = realtime_rag_evaluation(test_question)
result

reference가 없어 context_precision과 context_recall은 평가하지 않습니다.


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

RAGAS 점수:
  - Faithfulness: 1.000
  - Answer Relevancy: 0.519


{'question': 'RAGAS 평가 프레임워크의 핵심 지표는 무엇인가요?',
 'contexts': ['query', 'follow_up_questions', 'answer'],
 'answer': 'RAGAS 평가 프레임워크의 핵심 지표는 컨텍스트에 없습니다.',
 'scores': {'faithfulness': np.float64(1.0),
  'answer_relevancy': np.float64(0.5191700890364574),
  'context_precision': None,
  'context_recall': None}}

## RAGAS 평가

### RAGAS 4대 지표

1. Faithfulness (충실성): 답변의 주장이 컨텍스트로 지지되는 비율

2. Answer Relevancy (답변 관련성): 답변이 질문에 얼마나 관련 있는가

3. Context Precision (컨텍스트 정밀도): 검색된 컨텍스트 중 관련 있는 비율

4. Context Recall (컨텍스트 재현율): 정답에 필요한 정보가 컨텍스트에 포함된 비율



In [20]:
# RAGAS 평가용 데이터 변환
ragas_samples = []
for item in sample_data:
    contexts = normalize_contexts(item.get("contexts", []))
    ragas_samples.append(
        SingleTurnSample(
            user_input=item["question"],
            response=item.get("answer", ""),
            retrieved_contexts=contexts,
            reference=item.get("ground_truth"),
        )
    )

ragas_dataset = EvaluationDataset(samples=ragas_samples)
ragas_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=3)

In [21]:
# RAGAS 평가 실행
ragas_metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

ragas_result = evaluate(
    llm=llm,
    embeddings=embeddings,
    dataset=ragas_dataset,
    metrics=ragas_metrics,
)

# 집계 점수 출력
ragas_df = ragas_result.to_pandas()

for metric_name in ["faithfulness", "answer_relevancy", "context_precision", "context_recall"]:
    if metric_name in ragas_df.columns:
        avg_score = ragas_df[metric_name].mean()
        print(f"  - {metric_name}: {avg_score:.3f}")

ragas_df.head()

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


...

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  - faithfulness: 0.583
  - answer_relevancy: 0.743
  - context_precision: 1.000
  - context_recall: 0.667


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,RAG 시스템에서 Retriever의 역할은 무엇인가요?,[RAG 시스템의 Retriever 컴포넌트는 질문과 관련된 문서를 벡터 DB에서 ...,Retriever는 벡터 데이터베이스에서 질문과 관련된 문서를 검색하여 제공하는 컴...,Retriever는 사용자의 질문에 관련된 문서를 벡터 데이터베이스에서 검색하여 가...,1.00,0.788313,1.0,1.0
1,LangChain의 주요 장점은 무엇인가요?,[LangChain은 여러 LLM 프로바이더를 통합하여 사용할 수 있는 프레임워크입...,LangChain의 주요 장점은 다양한 LLM과 도구들을 손쉽게 연결하고 체인으로 ...,"LangChain은 LLM 애플리케이션 개발을 위한 통합 프레임워크로, 다양한 LL...",0.50,0.999999,1.0,1.0
2,Embedding 모델의 차원은 무엇을 의미하나요?,"[Embedding 차원이 높으면 더 세밀한 의미를 표현할 수 있습니다., Open...","차원은 벡터의 크기를 나타내며, 높을수록 의미를 더 정밀하게 표현하지만 메모리와 계...","Embedding 차원은 텍스트를 벡터로 변환할 때의 벡터 크기를 의미하며, 일반적...",0.25,0.441039,1.0,0.0


In [25]:
# 결과 통합
comparison_data = []
for i, item in enumerate(sample_data):
    row = {
        "question": item["question"][:50] + "...",
        "ragas_faithfulness": ragas_df.iloc[i].get("faithfulness", 0),
        "ragas_relevancy": ragas_df.iloc[i].get("answer_relevancy", 0),
        "ragas_ctx_precision": ragas_df.iloc[i].get("context_precision", 0),
        "ragas_ctx_recall": ragas_df.iloc[i].get("context_recall", 0),
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
comparison_df

,question,ragas_faithfulness,ragas_relevancy,ragas_ctx_precision,ragas_ctx_recall
0,RAG 시스템에서 Retriever의 역할은 무엇인가요?...,1.00,0.788313,1.0,1.0
1,LangChain의 주요 장점은 무엇인가요?...,0.50,0.999999,1.0,1.0
2,Embedding 모델의 차원은 무엇을 의미하나요?...,0.25,0.441039,1.0,0.0


In [26]:
# 통계
comparison_df[
    [
        "ragas_faithfulness",
        "ragas_relevancy",
        "ragas_ctx_precision",
        "ragas_ctx_recall",
    ]
].describe()

,ragas_faithfulness,ragas_relevancy,ragas_ctx_precision,ragas_ctx_recall
count,3.000000,3.000000,3.0,3.000000
mean,0.583333,0.743117,1.0,0.666667
std,0.381881,0.282208,0.0,0.577350
min,0.250000,0.441039,1.0,0.000000
25%,0.375000,0.614676,1.0,0.500000
50%,0.500000,0.788313,1.0,1.000000
75%,0.750000,0.894156,1.0,1.000000
max,1.000000,0.999999,1.0,1.000000


## 특이 케이스 분석

두 평가 방법 간의 불일치를 통해 개선 방향을 찾습니다.

In [27]:
# 케이스 1: Faithfulness 낮음
# → 의미적으로 유사하지만 컨텍스트 근거 부족 (잠재적 환각)

median_faith = comparison_df["ragas_faithfulness"].median()

low_faith = comparison_df[(comparison_df["ragas_faithfulness"] < median_faith)]

low_faith

,question,ragas_faithfulness,ragas_relevancy,ragas_ctx_precision,ragas_ctx_recall
2,Embedding 모델의 차원은 무엇을 의미하나요?...,0.25,0.441039,1.0,0.0


In [28]:
# 케이스 2: Relevancy 높음
# → 질문에 관련성 높지만 표현이 다름 (패러프레이즈)

median_rel = comparison_df["ragas_relevancy"].median()

high_rel = comparison_df[(comparison_df["ragas_relevancy"] >= median_rel)]

high_rel

,question,ragas_faithfulness,ragas_relevancy,ragas_ctx_precision,ragas_ctx_recall
0,RAG 시스템에서 Retriever의 역할은 무엇인가요?...,1.0,0.788313,1.0,1.0
1,LangChain의 주요 장점은 무엇인가요?...,0.5,0.999999,1.0,1.0


## 8. RAGAS 종합 평가 결과 해석

In [31]:
# RAGAS 종합 평가
avg_faith_val = comparison_df["ragas_faithfulness"].mean()
avg_rel_val = comparison_df["ragas_relevancy"].mean()
avg_ctx_p = comparison_df["ragas_ctx_precision"].mean()
avg_ctx_r = comparison_df["ragas_ctx_recall"].mean()

print("\nRAGAS 평균 점수:")
print(f"  - Faithfulness: {avg_faith_val:.3f}")
print(f"  - Relevancy: {avg_rel_val:.3f}")
print(f"  - Context Precision: {avg_ctx_p:.3f}")
print(f"  - Context Recall: {avg_ctx_r:.3f}")


# 항상 이러한 점수가 기준인 것은 아닙니다.
if avg_faith_val >= 0.8:
    print("높은 충실성 + 높은 유사도: 전반적으로 양호한 품질")
elif avg_faith_val < 0.7:
    print("낮은 충실성: 컨텍스트 근거 부족 또는 환각 위험")
    print("→ 프롬프트 개선, LLM 온도 낮추기, Guardrails 적용")

if avg_ctx_r < 0.7:
    print("낮은 Context Recall: 검색 실패 (필요 정보 누락)")
    print("→ top_k 증가, Hybrid 검색 (벡터 + BM25), 리트리버 성능 평가")

if avg_ctx_p < 0.7:
    print("낮은 Context Precision: 무관 문서 검색 (노이즈)")
    print("→ 리랭커 추가, 청킹 전략 재설계, 임베딩 모델 파인튜닝")


RAGAS 평균 점수:
  - Faithfulness: 0.583
  - Relevancy: 0.743
  - Context Precision: 1.000
  - Context Recall: 0.667

해석:
낮은 충실성: 컨텍스트 근거 부족 또는 환각 위험
→ 프롬프트 개선, LLM 온도 낮추기, Guardrails 적용
낮은 Context Recall: 검색 실패 (필요 정보 누락)
→ top_k 증가, Hybrid 검색 (벡터 + BM25), 리트리버 성능 평가
